In [8]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from jinja2 import Template
from pyne import data
from pyne.material import Material

/home/zoe/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)


First we want the helium density in $10^24$ atoms per cc.

\begin{align}
PV&=nRT\\
\frac{n}{V}&=\frac{P}{RT}
\end{align}

In [2]:

R = 8.314 # j/k-mol
temp = [250 ,750] #celsius
pres = [6*10**6, 5.84*10**6] #pascals
hedensity = np.zeros(len(temp))


for i, t in enumerate(temp):
    hedensity[i] = ((pres[i])/((t+273.15)*R)) # mol/m^3 

hedensavg = np.sum(hedensity)/len(hedensity)*(6.022*10**-7)# 10^24/cc

print(hedensavg)

0.000622077124202638


### Isotopic composition of the fuel

The enrichment of the UCO kernels in the TRISO particles is known.  Additionally, we know the molar contentrations of the three compounds that make UCO: $UO_2$ , $UC$, and $UC_{1.86}$.  The atomic mass of each element is given using the PYNE python library.  The steps are as follows:

\begin{align}
M_{UO_{2}} = (w_f)M_{235U} + (1-w_f)M_{238U} + (2M_{16O}) \\
M_{UC} = (w_f)M_{235U} + (1-w_f)M_{238U} + M_{12C} \\
M_{UC_{1.86}} = (w_f)M_{235U} + (1-w_f)M_{238U} + 1.86M_{12C} \\
\end{align}

The total mass of the UCO is the sum of these masses.  Then the isotopic concentration of each element is calculated as follows:

\begin{align}
w_{16O} = \frac{2\chi_{UO_2}M_{16O}}{M_{UCO}} \\
w_{235U} = \frac{(w_f)M_{235U}}{M_{UCO}} \\
w_{238U} = \frac{(1-w_f)M_{238U}}{M_{UCO}} \\
w_{12C} = \frac{\chi_{UC_{1.86}}(1.86)M_{12C} + \chi_{UC_{100}}M_{12C}}{M_{UCO}} \\
\end{align}

In [20]:
leu = Material({'U235': 0.155 , 'U238': 0.845})
uo2 = Material()
uc = Material()
uc186 = Material()

uo2.from_atom_frac({'O16': 2.0 , leu :1.0})
uc.from_atom_frac({'C12':1, leu: 1.0})
uc186.from_atom_frac({'C12':1.86, leu : 1.0})

uco = Material()
uco.from_atom_frac({uo2: 0.714, uc: 0.164, uc186: 0.123})
print(uco.mcnp()) #the negative values for composition mean it is mass fraction, just like serpent


m?
     6012 -1.7761e-02
     8016 -8.6071e-02
     92235 -1.3891e-01
     92238 -7.5726e-01



In [5]:
molUO2 + molUC186 + molUC100


1.001

In order to find the core dimensions, I took an image of the reactor core given by X-energy in a presentation to the NRC.  
<br>
Then, I opened it in an image viewer that gave the cursor's coordinates in pixels, measured the reactor core features in pixels.  Knowing the outer core diameter, developed a scale between pixels and meters, and used that to determine the core geometry.

\begin{align}
s (\frac{cm}{px}) = 100* \frac{RPV_D (m)}{RPV_D (px)}\\
x (cm) = x (px) * scale
\end{align}

In [6]:
core_dm = 4.88 # m
core_dpx = 69 #px
scale = core_dm/core_dpx *100 # cm/px

inner_corerad_px = 17.5
inner_corerad_cm = inner_corerad_px *scale
reflect_px = 13
reflect_cm = reflect_px *scale

print(2*(reflect_cm+inner_corerad_cm)) #can't be more than 488, which is the diameter including RPV
print()
print(inner_corerad_cm)
print(reflect_cm)
print(inner_corerad_cm+reflect_cm)

431.4202898550724

123.76811594202897
91.94202898550724
215.7101449275362


I found the midpoint of the cones on the top and bottom , and used the distance between these two midpoints as an adjusted height.  Then, I simplified by letting the reflector directly surround this region.

In [7]:
mid_bott_y = 228
mid_top_y = 91.5

adj_h_px = mid_bott_y - mid_top_y #px

adj_h_cm = adj_h_px*scale

print(adj_h_cm)

965.391304347826


### Creating the input file

The first step to creating the input file is to create the pebble and particle location input cards.  This can be done using serpent's built in particle dispersal routine, which takes the dimensions of the particle, the volume they are dispersed in, and the particle number or packing fraction, the universe the particles are in, and creates a plain text file, where each line is:

\begin{align}
x-coordinate\ \ y-coordinate\ \  z-coordinate\ \  universe
\end{align}

In [ ]:
#simple to read in a text file, but shouldnt store a variable tens or hundreds of thousands of elements long
# need to manipulate file without doing that (hopefully)
#readlines()

origin = open('pbbls.inp','r')

filenames = ['test1.txt','test2.txt']

lines = 0
for line in list(origin):
    lines += 1
origin.seek(0)
i = 0

while i < lines:
    for file in filenames:
        f = open(file,'a+')
        f.write(list(origin)[i] + 'a')
        origin.seek(0)
        f.close()
        i += 1
        if i == lines:
            break
            
origin.close()